# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [83]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [84]:
cities_data_file = "../WeatherPy/output_data/cities.csv"
cities_df = pd.read_csv(cities_data_file)
cities_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bay roberts,47.60,-53.26,37.40,100,90,8.05,CA,1603862103
1,laguna,38.42,-121.42,57.00,58,1,3.36,US,1603861947
2,hambantota,6.12,81.12,84.87,67,94,6.73,LK,1603861997
3,narsaq,60.92,-46.05,30.20,80,75,1.12,GL,1603862104
4,castro,-24.79,-50.01,55.54,76,0,3.20,BR,1603862104


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [98]:
# Get the locations and Humidity
locations = cities_df[["Lat","Lng"]].astype('float')
humidity = cities_df["Humidity"].astype('float')

gmaps.configure(api_key=g_key)

fig = gmaps.figure(center=(45.5051064,-122.6750261), zoom_level=2)

heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity,
    dissipating=False,
    max_intensity=100,
    point_radius = 1.25    
)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [106]:
ideal_vacation_weather = cities_df[
    (cities_df["Max Temp"] < 56) & 
    (cities_df["Max Temp"] > 44) &
    (cities_df["Wind Speed"] < 10) &
    (cities_df["Humidity"] < 50) &
    (cities_df["Cloudiness"] < 10)
]
ideal_vacation_weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
29,saint george,37.10,-113.58,51.80,18,1,4.16,US,1603861816
107,viedma,-40.81,-63.00,54.16,40,4,9.17,AR,1603862134
158,sayyan,15.17,44.32,53.08,35,3,5.86,YE,1603862150
162,urumqi,43.80,87.60,44.60,42,0,4.47,CN,1603862151
165,mehran,33.12,46.16,55.40,22,0,4.70,IR,1603862152
269,kedrovyy,56.17,91.82,48.20,45,0,8.03,RU,1603862185
284,hohhot,40.81,111.65,48.20,16,0,4.47,CN,1603861993
312,baherden,38.44,57.43,55.06,39,0,9.80,TM,1603862198
335,faya,18.39,42.45,55.40,30,0,5.82,SA,1603862205
388,kuhdasht,33.53,47.61,44.60,33,0,3.47,IR,1603862219


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [107]:
hotel_df = ideal_vacation_weather.copy(deep=True).reset_index(drop=True, inplace=False)
hotel_df["Hotel Name"] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,saint george,37.10,-113.58,51.80,18,1,4.16,US,1603861816,
1,viedma,-40.81,-63.00,54.16,40,4,9.17,AR,1603862134,
2,sayyan,15.17,44.32,53.08,35,3,5.86,YE,1603862150,
3,urumqi,43.80,87.60,44.60,42,0,4.47,CN,1603862151,
4,mehran,33.12,46.16,55.40,22,0,4.70,IR,1603862152,
5,kedrovyy,56.17,91.82,48.20,45,0,8.03,RU,1603862185,
6,hohhot,40.81,111.65,48.20,16,0,4.47,CN,1603861993,
7,baherden,38.44,57.43,55.06,39,0,9.80,TM,1603862198,
8,faya,18.39,42.45,55.40,30,0,5.82,SA,1603862205,
9,kuhdasht,33.53,47.61,44.60,33,0,3.47,IR,1603862219,


In [108]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "radius": 5000,
    "keyword": "hotel",
    "types": "lodging",
    "key": g_key,
}

for index, row in hotel_df.iterrows():
    
    lat = row['Lat']
    lng = row["Lng"]
    
    params['location'] = f"{lat},{lng}"
    
    response = requests.get(base_url, params=params)
    
    hotels = response.json()
    
    #pprint(hotels)
    
    try:
        hotel_name = hotels['results'][0]['name']
    except (KeyError, IndexError):
        hotel_name = "No Hotel Found"
        
    hotel_df.loc[index, "Hotel Name"] = hotel_name
    
hotel_df        

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,saint george,37.10,-113.58,51.80,18,1,4.16,US,1603861816,"The Advenire, Autograph Collection"
1,viedma,-40.81,-63.00,54.16,40,4,9.17,AR,1603862134,Hotel Austral Viedma
2,sayyan,15.17,44.32,53.08,35,3,5.86,YE,1603862150,No Hotel Found
3,urumqi,43.80,87.60,44.60,42,0,4.47,CN,1603862151,JinJiang International Hotel Urumqi
4,mehran,33.12,46.16,55.40,22,0,4.70,IR,1603862152,هتل رستوران میلاد
5,kedrovyy,56.17,91.82,48.20,45,0,8.03,RU,1603862185,No Hotel Found
6,hohhot,40.81,111.65,48.20,16,0,4.47,CN,1603861993,"Shangri-La Hotel, Hohhot"
7,baherden,38.44,57.43,55.06,39,0,9.80,TM,1603862198,No Hotel Found
8,faya,18.39,42.45,55.40,30,0,5.82,SA,1603862205,No Hotel Found
9,kuhdasht,33.53,47.61,44.60,33,0,3.47,IR,1603862219,Azadi Hotel


In [109]:

fig = gmaps.figure(center=(45.5051064,-122.6750261), zoom_level=1)

heat_layer = gmaps.heatmap_layer(
    locations,
    weights=humidity,
    dissipating=False,
    max_intensity=100,
    point_radius = 1.25    
)

fig.add_layer(heat_layer)

In [110]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
hotel_locations = hotel_df[["Lat", "Lng"]]

In [112]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.marker_layer(
    hotel_locations,
    info_box_content=hotel_info
)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))